In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop,Adam
import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from tensorflow.keras import layers,regularizers,Sequential,Model,backend,callbacks,optimizers,metrics,losses
import tensorflow as tf
import sys
import json
sys.path.append('../input/itration/iterative_stratification-0.1.6-py3-none-any.whl')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

df = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
non_ctl_idx = df.loc[df['cp_type']!='ctl_vehicle'].index.to_list()
df = df.drop(['sig_id','cp_type','cp_dose','cp_time'],axis=1)
data_X_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
non_ctl_data_X_test = data_X_test.loc[data_X_test['cp_type'] =='ctl_vehicle'].index.to_list()
data_X_test = data_X_test.drop(['sig_id','cp_type','cp_dose','cp_time'],axis=1)

pretrain = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
pretrain = pretrain.drop('sig_id',axis=1)
pretrain = pretrain.values
pretrain = pretrain[non_ctl_idx]

dada = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
dada = dada.drop('sig_id',axis=1)
df_label = dada.values
df = df.iloc[non_ctl_idx]
df_label = df_label[non_ctl_idx]



In [ ]:
qq = df.columns

scaler = preprocessing.MinMaxScaler()

data_train = scaler.fit_transform(df)

data_test = scaler.transform(data_X_test)

df = pd.DataFrame(data = data_train,columns = qq)
data_X_test = pd.DataFrame(data = data_test,columns = qq)

In [ ]:

json_file_path = '../input/mainfile/main_predictors.json'

with open(json_file_path, 'r') as j:
    predictors = json.loads(j.read())
    predictors = predictors['start_predictors']

In [ ]:


cs = df.columns.str.contains('c-')
gs = df.columns.str.contains('g-')

def preprocessor(train,test):
    
    
    n_gs = 2 
    n_cs = 100 
    
    pca_cs = PCA(n_components = n_cs)
    pca_gs = PCA(n_components = n_gs)

    train_pca_gs = pca_gs.fit_transform(train[:,gs])
    train_pca_cs = pca_cs.fit_transform(train[:,cs])
    test_pca_gs = pca_gs.transform(test[:,gs])
    test_pca_cs = pca_cs.transform(test[:,cs])

    
    train_c_mean = train[:,cs].mean(axis=1)
    test_c_mean = test[:,cs].mean(axis=1)
    train_g_mean = train[:,gs].mean(axis=1)
    test_g_mean = test[:,gs].mean(axis=1)
    
    
    train = np.concatenate((train,train_pca_gs,train_pca_cs,train_c_mean[:,np.newaxis]
                            ,train_g_mean[:,np.newaxis]),axis=1)
    test = np.concatenate((test,test_pca_gs,test_pca_cs,test_c_mean[:,np.newaxis],
                           test_g_mean[:,np.newaxis]),axis=1)
    
    scaler = preprocessing.StandardScaler()

    train = scaler.fit_transform(train)

    test = scaler.transform(test)
    
    return train, test




In [ ]:

p_min = 0.0005
p_max = 0.9995

def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -backend.mean(y_true*backend.log(y_pred) + (1-y_true)*backend.log(1-y_pred))

In [ ]:
def create_model(n_features, n_features_2, n_labels, label_smoothing = 0.0005):
    input_pri = tf.keras.layers.Input(shape = (n_features,), name = 'Input1') 
    input_sec = tf.keras.layers.Input(shape = (n_features_2,), name = 'Input2')
        
    mouth_1 = tf.keras.Sequential([
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation="relu"),
        ],name='Head1') 
    
    input_tir = mouth_1(input_pri)
    
    num_columns = 703
    aa = tf.keras.layers.Concatenate()([input_sec, input_tir]),
    
    mouth_2 = tf.keras.Sequential([
        tf.keras.layers.Input(num_columns),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(512, activation="relu")
        ],name='Head2')
    
    bb = mouth_2(aa)
    aa_1 = tf.keras.layers.Concatenate()([input_sec, input_tir ,bb]), 
    
    num_columns_1 = 1215
    
    stomach_1 = tf.keras.Sequential([
        tf.keras.layers.Input(num_columns_1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(512, activation="elu"),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Dense(256, activation="relu")  
    ],name='stomach_1')
    
    cc = stomach_1(aa_1)
    aa_2 = tf.keras.layers.Concatenate()([bb,cc]),    
    
    num_columns_2 = 768
    
    stomach_2 = tf.keras.Sequential([
        tf.keras.layers.Input(num_columns_2),
        tf.keras.layers.BatchNormalization(), 
        tf.keras.layers.Dense(256, activation="relu")  
    ],name='stomach_2')
    
    dd = stomach_2(aa_2)
    aa_3 = tf.keras.layers.Concatenate()([bb,cc,dd]),
    
    num_columns_3 = 1024
    
    
    stomach_3 = tf.keras.Sequential([
        tf.keras.layers.Input(num_columns_3),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, kernel_initializer='lecun_normal', activation='selu',name='last_frozen'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(206, kernel_initializer='lecun_normal', activation='selu')
        ],name='stomach_3')

    output_1 = stomach_3(aa_3)
    
    #output_2 = tf.keras.Sequential([
        #tf.keras.layers.BatchNormalization(),
        #tfa.layers.WeightNormalization(tf.keras.layers.Dense(206, activation="sigmoid"))  
    #],name = 'end')
    
    #output = output_2(output_1)
    
    
    model = Model(inputs = [input_pri, input_sec], outputs = output_1)
    return model

In [ ]:
tra, val = preprocessor(df.values,df.values)
trash = df.loc[:,predictors].values
model = create_model(tra.shape[1], trash.shape[1], df_label.shape[1])
model.summary()
tf.keras.utils.plot_model(model,show_shapes=True)

In [ ]:
import gc

from keras import backend 
n_seeds = 2
np.random.seed(1)
#seeds = np.random.randint(0,100,size=n_seeds)
#seeds = [23, 228, 1488, 1998, 2208, 2077, 404]
seeds = [23,1998]
n_labels = df_label.shape[1]
n_train = df.shape[0]
n_test = data_X_test.shape[0]
n_folds = 10
y_pred = np.zeros((n_test,n_labels))
oof = tf.constant(0.0)
hists = []


for seed in seeds:
    fold = 0
    print(fold)
    mlkf = KFold(n_splits=n_folds,shuffle=True,random_state=seed)
    for n,(train, test) in enumerate(mlkf.split(df,df_label)):
        tf.print('OOF score is ',oof)
        print(f'Fold {n}')
        
        file_name = ('model weights of seed '+ str(seed)+' and fold '+str(fold)+'.h5')
        
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.5, verbose=1,mode='auto',patience=2, min_lr=1E-5)
        early_st = tf.keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=6, verbose=1, mode='auto',baseline=None, restore_best_weights=True)
        callbacks = tf.keras.callbacks.ModelCheckpoint(filepath = file_name, monitor='val_logloss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto')
        
        
        train_pri, validation_pri = preprocessor(df.iloc[train].values,
                                       df.iloc[test].values)
        _,data_test_pri = preprocessor(df.iloc[train].values,
                                   data_X_test.values)
        
        train_sec = df.iloc[train][predictors].values
        validation_sec = df.iloc[test][predictors].values
        data_test_sec = data_X_test[predictors].values
        
        pretrain_y = pretrain[train]
        prevalidation_y = pretrain[test]

        train_y = df_label[train]
        validation_y = df_label[test]
        
        n_features = train_pri.shape[1]
        n_features_2 = train_sec.shape[1]
        
        ##pretraing
        n_labels_pre = pretrain.shape[1]
        
        tf.keras.callbacks.Callback()
            
        base_model = create_model(n_features, n_features_2, n_labels_pre)
        
        output_2 = tf.keras.Sequential([
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(n_labels_pre, activation="sigmoid")],name = 'end')
        
        output_2_2 = output_2(base_model.output)
        
        model = Model(base_model.input,output_2_2)
     
        model.compile(optimizer=tf.optimizers.Adam(),loss=losses.BinaryCrossentropy(label_smoothing=0.0005),metrics=logloss)

        hist = model.fit([train_pri,train_sec],pretrain_y, batch_size=128, epochs=200,verbose=1,validation_data = ([validation_pri,validation_sec],prevalidation_y),callbacks =[reduce_lr,callbacks,early_st])
        
        model.load_weights(file_name)
        
        loss = model.evaluate([validation_pri,validation_sec], prevalidation_y, verbose=2)  
        
        
        ##maintraing
        
        file_name_1 = ('model weights of seed '+ str(seed)+' and fold '+str(fold)+'.h5')
        
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=1,mode='auto',patience=2, min_lr=1E-5)
        early_st = tf.keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=3, verbose=1, mode='auto',baseline=None, restore_best_weights=True)
        callbacks = tf.keras.callbacks.ModelCheckpoint(filepath = file_name_1, monitor='val_logloss', verbose=0, save_best_only=True, save_weights_only=True, mode='auto')
        
        
        n_labels_pre = train_y.shape[1]
        
        new_model = create_model(n_features, n_features_2, n_labels_pre)

        sz = len(base_model.layers)
        for k in range(sz):
            w = model.layers[k].get_weights()
            new_model.layers[k].set_weights(w)
        
        output_3 = tf.keras.Sequential([
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(n_labels_pre, activation="sigmoid")])
        
        output_33 = output_3(new_model.output)
        
        
        model_1 = Model(new_model.input,output_33)
        
        model_1.summary()
     
        model_1.compile(optimizer=tf.optimizers.Adam(),loss=losses.BinaryCrossentropy(label_smoothing=0.0005),metrics=logloss)

        hist = model_1.fit([train_pri,train_sec],train_y, batch_size=128, epochs=200,verbose=1,validation_data = ([validation_pri,validation_sec],validation_y),callbacks =[reduce_lr,callbacks,early_st])
        
        model_1.load_weights(file_name_1)
        
        loss = model_1.evaluate([validation_pri,validation_sec], validation_y, verbose=2)  
                     
        
        
        ## finetraining 
        
        for layer in model_1.layers[:11]:
            layer.trainable = False
            
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_logloss', factor=0.1, verbose=1,mode='auto',patience=2, min_lr=1E-5)
        early_st = tf.keras.callbacks.EarlyStopping(monitor='val_logloss', min_delta=1E-5, patience=4, verbose=1, mode='auto',baseline=None, restore_best_weights=True)
            
        model_1.compile(optimizer=tf.optimizers.Adadelta(learning_rate=0.001/3),loss=losses.BinaryCrossentropy(label_smoothing=0.0005),metrics=logloss)

        hist = model_1.fit([train_pri,train_sec],train_y, batch_size=128, epochs=200,verbose=1,validation_data = ([validation_pri,validation_sec],validation_y),callbacks =[reduce_lr,callbacks,early_st])
            
    
        
        # Save Model
        model_1.save('_seed_'+str(seed)+'_fold_'+str(fold))

        # OOF Score
        y_val = model_1.predict([validation_pri,validation_sec])
        oof += logloss(tf.constant(validation_y,dtype=tf.float32),tf.constant(y_val,dtype=tf.float32))/(n_folds*n_seeds)

        # Run prediction
        y_pred += model_1.predict([data_test_pri,data_test_sec])/(n_folds*n_seeds)
        
        fold += 1
        
        gc.collect()
        tf.keras.backend.clear_session()
        del train_pri
        del train_sec
        del validation_pri
        del validation_sec
        del train_y
        del validation_y
        del data_test_pri
        del data_test_sec
        
        


In [ ]:
tf.print('OOF score is ',oof)



In [ ]:
tf.keras.utils.plot_model(model_1,show_shapes=True)

In [ ]:

final = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
final.iloc[:,1:] = np.clip(y_pred,p_min,p_max)
final.iloc[non_ctl_data_X_test,1:] = 0


final.to_csv('submission.csv', index=False)
final.head()